In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd

# 1.데이터 로드

In [2]:
data_path = '../data'
# data_path = '/Users/pervin0527/upstage-ml-regression-ml11/data'
train_path = f'{data_path}/new/new_train_ver2.csv'
test_path  = f'{data_path}/new/new_test_ver2.csv'

print(os.listdir(data_path))

['new', 'subway_feature.csv', 'train.csv', 'test.csv', 'bus_feature.csv', 'sample_submission.csv']


In [3]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
print('Train data shape : ', train_data.shape, 'Test data shape : ', test_data.shape)

Train data shape :  (1118822, 52) Test data shape :  (9272, 51)


## 1-1.병합

In [4]:
# train/test 구분을 위한 칼럼 생성.
train_data['is_test'] = 0
test_data['is_test'] = 1
data = pd.concat([train_data, test_data]) # 하나의 데이터로 합친다.

print(data.shape)

(1128094, 53)


# 2.컬럼 제거

In [5]:
## 제거할컬럼
drop_columns = [
    'k-사용검사일-사용승인일', 
    'k-등록일자',
    'k-수정일자', 
    '단지승인일', 
    '단지신청일',
    '중개사소재지', 
    'k-전화번호', 
    'k-팩스번호', 
    '단지소개기존clob', 
    'k-홈페이지', 
    '고용보험관리번호', 
    '기타/의무/임대/임의=1/2/3/4', 
    '사용허가여부', 
    '관리비 업로드',
    'k-전용면적별세대현황(60㎡이하)', 
    'k-전용면적별세대현황(60㎡~85㎡이하)',  
    'k-85㎡~135㎡이하', 
    'k-135㎡초과'
]
data = data.drop(drop_columns, axis=1)
print(data.shape)

(1128094, 35)


In [6]:
## 결측치를 확인.
data.isnull().sum()

시군구                         0
번지                          0
본번                          0
부번                          0
아파트명                     2136
전용면적(㎡)                     0
계약년월                        0
계약일                         0
층                           0
건축년도                        0
도로명                         0
해제사유발생일               1121899
등기신청일자                      0
거래유형                        0
k-단지분류(아파트,주상복합등등)     877273
k-세대타입(분양형태)           876125
k-관리방식                 876125
k-복도유형                 876454
k-난방방식                 876125
k-전체동수                 877207
k-전체세대수                876125
k-건설사(시공사)             877637
k-시행사                  877834
k-연면적                  876125
k-주거전용면적               876170
k-관리비부과면적              876125
경비비관리형태                877561
세대전기계약방법               885389
청소비관리형태                877751
건축면적                   876279
주차대수                   876277
좌표X                         0
좌표Y                         0
target    

# 3.파생변수

## 3-0.전용면적 rename

In [7]:
data = data.rename(columns={'전용면적(㎡)':'전용면적'})

In [8]:
data.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', 'k-단지분류(아파트,주상복합등등)',
       'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
       'k-건설사(시공사)', 'k-시행사', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', '경비비관리형태',
       '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '좌표X', '좌표Y', 'target',
       'is_test'],
      dtype='object')

## 3-1."시군구" $\rightarrow$ "시", "군", "구"로 분할

In [9]:
# '시군구' 컬럼을 공백을 기준으로 분할
data[['시', '구', '동']] = data['시군구'].str.split(' ', expand=True)
data = data.drop(['시'], axis=1)

data.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', 'k-단지분류(아파트,주상복합등등)',
       'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
       'k-건설사(시공사)', 'k-시행사', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', '경비비관리형태',
       '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '좌표X', '좌표Y', 'target',
       'is_test', '구', '동'],
      dtype='object')

In [10]:
print(len(data['구'].unique()))
print(data['구'].unique())

25
['강남구' '강동구' '강북구' '강서구' '관악구' '광진구' '구로구' '금천구' '노원구' '도봉구' '동대문구' '동작구'
 '마포구' '서대문구' '서초구' '성동구' '성북구' '송파구' '양천구' '영등포구' '용산구' '은평구' '종로구' '중구'
 '중랑구']


In [11]:
print(len(data['동'].unique()))
print(data['동'].unique())

337
['개포동' '논현동' '대치동' '도곡동' '삼성동' '세곡동' '수서동' '신사동' '역삼동' '압구정동' '일원동' '자곡동'
 '청담동' '강일동' '율현동' '고덕동' '길동' '둔촌동' '명일동' '상일동' '성내동' '암사동' '천호동' '미아동'
 '번동' '가양동' '수유동' '우이동' '내발산동' '등촌동' '공항동' '마곡동' '방화동' '염창동' '화곡동' '남현동'
 '봉천동' '신림동' '광장동' '구의동' '자양동' '군자동' '화양동' '개봉동' '고척동' '중곡동' '가리봉동' '구로동'
 '오류동' '천왕동' '독산동' '온수동' '궁동' '신도림동' '항동' '가산동' '시흥동' '공릉동' '상계동' '월계동'
 '중계동' '하계동' '도봉동' '방학동' '쌍문동' '창동' '답십리동' '용두동' '이문동' '신설동' '장안동' '전농동'
 '회기동' '제기동' '청량리동' '휘경동' '노량진동' '대방동' '동작동' '본동' '사당동' '상도동' '상도1동'
 '신대방동' '흑석동' '공덕동' '노고산동' '대흥동' '도화동' '마포동' '상수동' '상암동' '서교동' '성산동'
 '신공덕동' '신수동' '신정동' '아현동' '동교동' '망원동' '염리동' '용강동' '중동' '창전동' '하중동' '합정동'
 '남가좌동' '냉천동' '북가좌동' '연남동' '토정동' '현석동' '대현동' '연희동' '영천동' '충정로3가' '현저동'
 '홍은동' '홍제동' '북아현동' '창천동' '천연동' '충정로2가' '합동' '내곡동' '반포동' '방배동' '서초동' '신원동'
 '우면동' '잠원동' '양재동' '금호동1가' '금호동2가' '금호동3가' '성수동1가' '성수동2가' '금호동4가' '송정동'
 '도선동' '마장동' '사근동' '상왕십리동' '옥수동' '하왕십리동' '행당동' '응봉동' '용답동' '홍익동' '길음동'
 '돈암동' '동소문동5가' '동소문동7가' '보문동3가' '동선동1가' '동선동3가' '동선동4가' 

## 3-2.계약일 drop

In [12]:
# data['계약날짜'] = pd.to_datetime(data['계약년월'].astype(str) + data['계약일'].astype(str), format='%Y%m%d')
# data = data.drop(columns=['계약년월', '계약일'])
# data.head(3)

data = data.drop(columns=['계약일'])

## 3-3.해제사유발생여부, 등기신청여부 생성

In [13]:
# data['해제사유발생여부'] = pd.notnull(data['해제사유발생일']).astype(int)
# print(data[['해제사유발생여부']])

In [14]:
# data['등기신청일자여부'] = pd.notnull(data['등기신청일자']).astype(int)
# print(data[['등기신청일자여부']])

In [15]:
# data = data.drop(columns=['해제사유발생일', '등기신청일자'])

## 3-4.세대별 주차대수

In [16]:
data['세대별_주차대수'] = data['주차대수'] / data['k-전체세대수']
data = data.drop(columns=['주차대수'])

# 4.범주형 변수 결측치 처리

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1128094 entries, 0 to 9271
Data columns (total 36 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   시군구                 1128094 non-null  object 
 1   번지                  1128094 non-null  object 
 2   본번                  1128094 non-null  float64
 3   부번                  1128094 non-null  float64
 4   아파트명                1125958 non-null  object 
 5   전용면적                1128094 non-null  float64
 6   계약년월                1128094 non-null  int64  
 7   층                   1128094 non-null  int64  
 8   건축년도                1128094 non-null  int64  
 9   도로명                 1128094 non-null  object 
 10  해제사유발생일             6195 non-null     float64
 11  등기신청일자              1128094 non-null  object 
 12  거래유형                1128094 non-null  object 
 13  k-단지분류(아파트,주상복합등등)  250821 non-null   object 
 14  k-세대타입(분양형태)        251969 non-null   object 
 15  k-관리방식              251

## 4-1.k-세대타입(분양형태)

- 개인이 분양인이지, 임대인지 계약 정보를 대체할 수 없기 때문에 ```nan```은 ```unknown```으로 처리한다. 
- 컬럼 이름을 ```분양형태```로 대체한다.
- 결측치를 채운다면 **모델 예측**을 예상하고 있다.

In [18]:
display(data['k-세대타입(분양형태)'].unique())

array(['분양', '기타', nan, '임대'], dtype=object)

In [19]:
data['k-세대타입(분양형태)'] = data['k-세대타입(분양형태)'].fillna('unknown')
data['k-세대타입(분양형태)'] = data['k-세대타입(분양형태)'].replace('기타', 'unknown')
data = data.rename(columns={'k-세대타입(분양형태)':'분양형태'})
display(data['분양형태'].value_counts())

분양형태
unknown    917763
분양         208098
임대           2233
Name: count, dtype: int64

## 4-2.k-단지분류(아파트,주상복합등등)

- 건물이 [아파트, 주상복합, 연립주택, 도시형 생활주택(아파트), 도시형 생활주택(주상복합), nan] 어떤 유형인지 나타냄.
- nan을 ```unknown```으로 대체
- 컬럼 이름을 ```부동산유형```으로 변경한다.

In [20]:
display(data['k-단지분류(아파트,주상복합등등)'].unique())

array(['아파트', '주상복합', nan, '연립주택', '도시형 생활주택(아파트)', '도시형 생활주택(주상복합)'],
      dtype=object)

In [21]:
data['k-단지분류(아파트,주상복합등등)'] = data['k-단지분류(아파트,주상복합등등)'].fillna('unknown')
data = data.rename(columns={'k-단지분류(아파트,주상복합등등)':'부동산유형'})

print(data['부동산유형'].value_counts())

부동산유형
unknown           877273
아파트               238586
주상복합               11450
도시형 생활주택(주상복합)       505
도시형 생활주택(아파트)        155
연립주택                 125
Name: count, dtype: int64


## 4-3.k-관리방식, k-복도유형, k-난방방식

In [22]:
display(data['k-관리방식'].unique())

array(['자치관리', '위탁관리', '직영', nan], dtype=object)

In [23]:
data['k-관리방식'] = data['k-관리방식'].fillna('unknown')
print(data['k-관리방식'].value_counts())

k-관리방식
unknown    876125
위탁관리       230361
자치관리        18144
직영           3464
Name: count, dtype: int64


In [24]:
display(data['k-복도유형'].unique())

array(['계단식', '혼합식', '복도식', '타워형', '기타', nan], dtype=object)

In [25]:
data['k-복도유형'] = data['k-복도유형'].fillna('unknown')
data['k-복도유형'] = data['k-복도유형'].replace('기타', 'unknown')
print(data['k-복도유형'].value_counts())

k-복도유형
unknown    877536
계단식        124964
혼합식         85513
복도식         38710
타워형          1371
Name: count, dtype: int64


In [26]:
display(data['k-난방방식'].unique())

array(['개별난방', '지역난방', '기타', nan, '중앙난방'], dtype=object)

In [27]:
data['k-난방방식'] = data['k-난방방식'].fillna('unknown')
data['k-난방방식'] = data['k-난방방식'].replace('기타', 'unknown')
print(data['k-난방방식'].value_counts())

k-난방방식
unknown    878216
개별난방       152365
지역난방        84826
중앙난방        12687
Name: count, dtype: int64


## 4-4.경비비관리형태, 세대전기계약방법, 청소비관리형태

In [28]:
print(data["경비비관리형태"].unique())

['직영' '위탁' '위탁+직영' nan '기타']


In [29]:
data["경비비관리형태"] = data["경비비관리형태"].fillna('unknown')
data["경비비관리형태"] = data["경비비관리형태"].replace("기타", "unknown")
print(data["경비비관리형태"].value_counts())
print(data["경비비관리형태"].unique())

경비비관리형태
unknown    879378
위탁         208757
직영          33791
위탁+직영        6168
Name: count, dtype: int64
['직영' '위탁' '위탁+직영' 'unknown']


In [30]:
print(data["세대전기계약방법"].unique())

['단일계약' '종합계약' nan]


In [31]:
data["세대전기계약방법"] = data["세대전기계약방법"].fillna('unknown')
print(data["세대전기계약방법"].value_counts())

세대전기계약방법
unknown    885389
종합계약       124610
단일계약       118095
Name: count, dtype: int64


In [32]:
print(data["청소비관리형태"].unique())

['직영' '위탁' '위탁+직영' nan '기타']


In [33]:
data["청소비관리형태"] = data["청소비관리형태"].fillna('unknown')
data["청소비관리형태"] = data["청소비관리형태"].replace("기타", "unknown")
print(data["청소비관리형태"].value_counts())
print(data["청소비관리형태"].unique())

청소비관리형태
unknown    881230
위탁         227511
직영          15174
위탁+직영        4179
Name: count, dtype: int64
['직영' '위탁' '위탁+직영' 'unknown']


## 4-5.아파트명 데이터 정리

- 숫자로만 구성되어 있거나 아파트명이 없는 경우 숫자-숫자 형태로 채워진 데이터가 존재.
- 201호 미공시와 같은 의미없는 값도 존재.
- nan
- 이들을 모두 unknown으로 처리

In [34]:
def convert_to_unknown(x):
    if pd.isna(x):
        return 'unknown'
    if not isinstance(x, str):
        return 'unknown'
    if x.isdigit():
        return 'unknown'
    if '-' in x and all(part.isdigit() for part in x.split('-')):
        return 'unknown'
    return x

data['아파트명'] = data['아파트명'].apply(convert_to_unknown)
data['아파트명'] = data['아파트명'].replace('201호 미공시', 'unknown')

# 5.처리된 데이터 분할 및 저장

In [35]:
data = data.drop(columns=['k-복도유형', 'k-관리방식', 'k-난방방식', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
                          '본번', '부번', '구', '동', '부동산유형', '분양형태', 'k-전체동수', 'k-전체세대수', 'k-시행사', 
                          'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', '건축면적'])

In [36]:
train_df = data[data['is_test'] == 0]
test_df = data[data['is_test'] == 1]

train_df = train_df.drop(columns=['is_test'])
test_df = test_df.drop(columns=['is_test', 'target'])

train_df.to_csv(f'{data_path}/new/new_train_ver3.csv', index=False)
test_df.to_csv(f'{data_path}/new/new_test_ver3.csv', index=False)

In [37]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1118822 entries, 0 to 1118821
Data columns (total 16 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   시군구         1118822 non-null  object 
 1   번지          1118822 non-null  object 
 2   아파트명        1118822 non-null  object 
 3   전용면적        1118822 non-null  float64
 4   계약년월        1118822 non-null  int64  
 5   층           1118822 non-null  int64  
 6   건축년도        1118822 non-null  int64  
 7   도로명         1118822 non-null  object 
 8   해제사유발생일     5983 non-null     float64
 9   등기신청일자      1118822 non-null  object 
 10  거래유형        1118822 non-null  object 
 11  k-건설사(시공사)  247764 non-null   object 
 12  좌표X         1118822 non-null  float64
 13  좌표Y         1118822 non-null  float64
 14  target      1118822 non-null  float64
 15  세대별_주차대수    249108 non-null   float64
dtypes: float64(6), int64(3), object(7)
memory usage: 145.1+ MB


In [38]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9272 entries, 0 to 9271
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   시군구         9272 non-null   object 
 1   번지          9272 non-null   object 
 2   아파트명        9272 non-null   object 
 3   전용면적        9272 non-null   float64
 4   계약년월        9272 non-null   int64  
 5   층           9272 non-null   int64  
 6   건축년도        9272 non-null   int64  
 7   도로명         9272 non-null   object 
 8   해제사유발생일     212 non-null    float64
 9   등기신청일자      9272 non-null   object 
 10  거래유형        9272 non-null   object 
 11  k-건설사(시공사)  2693 non-null   object 
 12  좌표X         9272 non-null   float64
 13  좌표Y         9272 non-null   float64
 14  세대별_주차대수    2709 non-null   float64
dtypes: float64(5), int64(3), object(7)
memory usage: 1.1+ MB
